# TF-IDF Vectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd

toy_corpus = ["the fat cat sat on the mat", "the big cat slept", "the dog chased a cat"]
vectorizer = TfidfVectorizer(use_idf=True)

corpus_tfidf = vectorizer.fit_transform(toy_corpus)

print(f"The vocabulary size is {len(vectorizer.vocabulary_.keys())} ")
print(f"The document-term matrix shape is {corpus_tfidf.shape}")

df = pd.DataFrame(np.round(corpus_tfidf.toarray(), 2))
df.columns = vectorizer.get_feature_names()
df

## Classification

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC

labels = [0, 1, 0]
clf = SVC()
clf.fit(df, labels)

In [ ]:
clf.predict(df)

# Building a LM Model 
Once we prepared our corpus above, we are ready to start training Maximum Likelihood Estimator (MLE) as a Language Model.

## Training a bigram LM

In [ ]:
!pip install nltk==3.5.0

In [ ]:
import nltk
from nltk.corpus import gutenberg
from nltk.lm import MLE
from nltk.lm.preprocessing import padded_everygram_pipeline

nltk.download("gutenberg")
nltk.download("punkt")
macbeth = gutenberg.sents("shakespeare-macbeth.txt")

model, vocab = padded_everygram_pipeline(2, macbeth)
lm = MLE(2)
lm.fit(model, vocab)
print(list(lm.vocab)[:40])
print(f"The number of words is {len(lm.vocab)}")

In [ ]:
print(macbeth[42])

## See what LM learned

Here is a list of what the language model learded so far

In [ ]:
print(f"The frequency of the term 'Macbeth' is {lm.counts['Macbeth']}")
print(f"The language model probability score of 'Macbeth' is {lm.score('Macbeth')}")
print(
    f"The number of times 'Macbeth' follows 'Enter' is {lm.counts[['Enter']]['Macbeth']} "
)
print(f"P(Macbeth | Enter) is {lm.score('Macbeth', ['Enter'])}")
print(f"P(shaking | for) is {lm.score('shaking', ['for'])}")

## Language Generation with LM

To generate one word

In [ ]:
lm.generate(1, random_seed=42)

To generate a sentence of 7 words length

In [ ]:
print(lm.generate(7, random_seed=42))

To generate 10 words starting with \<s>

In [ ]:
lm.generate(10, text_seed=["<s>"], random_seed=42)

# Word Embeddings Training

In [ ]:
!pip install gensim==3.8.3

In [ ]:
from gensim.models import Word2Vec

model = Word2Vec(
    sentences=macbeth, size=100, window=4, min_count=10, workers=4, iter=10
)

In [ ]:
model.wv.similar_by_word("then", 10)

In [ ]:
model.wv["did"]  # get numpy vector of word 'Macbeth'

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import random
import numpy as np

np.random.seed(42)
words = list([e for e in model.wv.vocab if len(e) > 4])  # plot words longer than 4
random.shuffle(words)
words3d = PCA(n_components=3, random_state=42).fit_transform(model.wv[words[:100]])


def plotWords3D(vecs, words, title):
    """
    Parameters
    ----------
    vecs : numpy-array
        Transformed 3D array either by PCA or other techniques
    words: a list of word
        the word list to be mapped
    title: str
        The title of plot
    """
    fig = plt.figure(figsize=(14, 10))
    ax = fig.gca(projection="3d")
    for w, vec in zip(words, vecs):
        ax.text(
            vec[0],
            vec[1],
            vec[2],
            w,
            color=np.random.rand(
                3,
            ),
        )
    ax.set_xlim(min(vecs[:, 0]), max(vecs[:, 0]))
    ax.set_ylim(min(vecs[:, 1]), max(vecs[:, 1]))
    ax.set_zlim(min(vecs[:, 2]), max(vecs[:, 2]))
    ax.set_xlabel("DIM-1")
    ax.set_ylabel("DIM-2")
    ax.set_zlabel("DIM-3")
    plt.title(title)
    plt.show()


plotWords3D(words3d, words, "Visualizing Word2Vec Word Embeddings using PCA")

## FastText

In [ ]:
!pip install fasttext

In [ ]:
# Parameters min_n and max_n take control the lengths of character ngrams.
# If max_n is set to 0, no character ngrams are used, and the model turns out to be Word2Vec.

In [ ]:
from gensim.models import FastText

model = FastText(
    sentences=macbeth,
    size=100,
    window=5,
    min_count=5,
    workers=4,
    iter=10,
    word_ngrams=3,
)
# min_n: min length of char ngrams (Default 3)
# max_n: max length of char ngrams (Default 6)

np.random.seed(42)
words = [w[0] for w in model.wv.similar_by_word("Macbeth", 50)]
words3d = PCA(n_components=3, random_state=42).fit_transform(model.wv[words])
plotWords3D(words3d, words, "Visualizing FastText Word Embeddings reduced by PCA")

In [ ]:
## Glove

In [ ]:
import gensim.downloader as api

info = api.info()  # show info about available models/datasets
model = api.load(
    "glove-twitter-25"
)  # download the model and return as object ready for use

In [ ]:
model.most_similar("java")

# LSTM Training

In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
!wget https://dl.fbaipublicfiles.com/glue/data/SST-2.zip
!unzip SST-2.zip

In [ ]:
import pandas as pd

df = pd.read_csv("SST-2/train.tsv", sep="\t")
sentences = df.sentence
labels = df.label

In [ ]:
df.head(3)

In [ ]:
max_sen_len = max([len(s.split()) for s in sentences])
words = ["PAD"] + list(set([w for s in sentences for w in s.split()]))
word2idx = {w: i for i, w in enumerate(words)}
max_words = max(word2idx.values()) + 1
idx2word = {i: w for i, w in enumerate(words)}
# preparing training set
train = [list(map(lambda x: word2idx[x], s.split())) for s in sentences]

In [ ]:
len(train), len(labels)

In [ ]:
sum(labels), len(labels), sum(labels) / len(labels)

In [ ]:
from keras import preprocessing

train_pad = preprocessing.sequence.pad_sequences(train, maxlen=max_sen_len)
print("train shape:", train_pad.shape)

In [ ]:
print(train_pad)

In [ ]:
from keras.layers import LSTM, Embedding, Dense
from keras.models import Sequential

model = Sequential()
model.add(Embedding(max_words, 32))
model.add(LSTM(32))
model.add(Dense(1, activation="sigmoid"))

model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["acc"])
history = model.fit(train_pad, labels, epochs=15, batch_size=16, validation_split=0.2)

In [ ]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history["val_" + string])
    plt.xlabel("epochs")
    plt.ylabel(string)
    plt.legend([string, "val_" + string])
    plt.show()


plot_graphs(history, "acc")
plot_graphs(history, "loss")

In [ ]:
from keras import layers

model = Sequential()
model.add(layers.Embedding(max_words, 32, input_length=max_sen_len))
model.add(layers.Conv1D(32, 8, activation="relu"))
model.add(layers.MaxPooling1D(4))
model.add(layers.Conv1D(32, 3, activation="relu"))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1, activation="sigmoid"))
model.summary()
model.compile(loss="binary_crossentropy", metrics=["acc"])
history = model.fit(train_pad, labels, epochs=15, batch_size=32, validation_split=0.2)

In [ ]:
plot_graphs(history, "acc")
plot_graphs(history, "loss")